In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine, text
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
data = pd.read_sql_query('select * from houseprices',con=engine)
# No need for an open connection, 
# because you're only doing a single query
engine.dispose()

# Step 2: Data Cleaning
# Handling missing values, encoding categorical variables, scaling numerical variables

# Identify columns with missing values
missing_values = data.isnull().sum()
missing_values.info()
# Impute missing values for numerical columns with median
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[num_cols] = data[num_cols].fillna(data[num_cols].median())

# Impute missing values for categorical columns with mode
cat_cols = data.select_dtypes(include=['object']).columns
for col in cat_cols:
    if data[col].isnull().any():
        mode_value = data[col].mode()
        if not mode_value.empty:
            data[col].fillna(mode_value[0], inplace=True)

# Encoding categorical variables using OneHotEncoder
data = pd.get_dummies(data, columns=cat_cols, drop_first=True)

# Split the data into training and test sets
X = data.drop('saleprice', axis=1)
y = data['saleprice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Decision Tree model
start_time = time.time()
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(X_train, y_train)
dt_predictions = decision_tree.predict(X_test)
dt_runtime = time.time() - start_time

# Evaluate Decision Tree model
dt_mse = mean_squared_error(y_test, dt_predictions)
dt_r2 = r2_score(y_test, dt_predictions)

# Train a Random Forest model
start_time = time.time()
random_forest = RandomForestRegressor(n_estimators=10, random_state=42)  # Simplest random forest
random_forest.fit(X_train, y_train)
rf_predictions = random_forest.predict(X_test)
rf_runtime = time.time() - start_time

# Evaluate Random Forest model
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print("Decision Tree - Runtime:", dt_runtime, "MSE:", dt_mse, "R2:", dt_r2)
print("Random Forest - Runtime:", rf_runtime, "MSE:", rf_mse, "R2:", rf_r2)


<class 'pandas.core.series.Series'>
Index: 81 entries, id to saleprice
Series name: None
Non-Null Count  Dtype
--------------  -----
81 non-null     int64
dtypes: int64(1)
memory usage: 3.3+ KB
Decision Tree - Runtime: 0.8694448471069336 MSE: 1574621056.0068493 R2: 0.7879894690619006
Random Forest - Runtime: 4.612690448760986 MSE: 1049492428.6264043 R2: 0.8586939720132692


### Model Comparison

### Decision Tree:

- Runtime: 0.8694448471069336 seconds
- Mean Squared Error (MSE): 1574621056.0068493
- R-squared (R2): 0.7879894690619006

### Random Forest:

- Runtime: 4.612690448760986 seconds
- Mean Squared Error (MSE): 1049492428.6264043
- R-squared (R2): 0.8586939720132692

### Analysis

The results show that the Random Forest model significantly outperforms the Decision Tree model in terms of predictive accuracy. The Random Forest has a lower Mean Squared Error (MSE) and a higher R-squared (R2) value compared to the Decision Tree, indicating better performance.

However, this improved performance comes at a cost. The runtime for the Random Forest model is substantially higher than that of the Decision Tree. The increase in computational complexity and runtime is a trade-off for the enhanced predictive power.